# Day 18: Boiling Boulders

The scan approximates the shape of the lava droplet with 1x1x1 cubes on a 3D grid, each given as its x,y,z position. To approximate the surface area, count the number of sides of each cube that are not immediately connected to another cube. So, if your scan were only two adjacent cubes like 1,1,1 and 2,1,1, each cube would have a single side covered and five sides exposed, a total surface area of 10 sides.


In [1]:
example = True

if example:
    puzzle = '''2,2,2
1,2,2
3,2,2
2,1,2
2,3,2
2,2,1
2,2,3
2,2,4
2,2,6
1,2,5
3,2,5
2,1,5
2,3,5'''
    
else:
    with open('data/obsidian.txt', 'r') as f:
        puzzle = f.read()    

### Strategy (part 1)
If we hold one dimension constant, we're looking at a plane. If we hold 2 dimensions constant, we're looking at all the blocks in a row (along any axis). 

For any number of blocks that are stuck together, in the row under consideration there are only the faces at the ends. If there are any gaps in the row, two additional faces appear for each gap.

We'll organize the coordinates in a dataframe so we can use the groupby method (split, apply, combine) to select rows of blocks. We will write a custom aggregation function to count the exposed faces in each of those rows.

In [2]:
import pandas as pd

cubes = []
for string in puzzle.split('\n'):
    coord = string.split(',')
    cube = (int(coord[0]), int(coord[1]), int(coord[2]))
    cubes.append(cube)
    
cubes = pd.DataFrame(cubes, columns = ['x', 'y', 'z'])
cubes = cubes.sort_values(by=['x', 'y', 'z'])

cubes

,x,y,z
1,1,2,2
9,1,2,5
3,2,1,2
11,2,1,5
5,2,2,1
0,2,2,2
6,2,2,3
7,2,2,4
8,2,2,6
4,2,3,2


In [3]:
def count_edges(cubes, axis):
    integers = cubes[[axis]].values
    spaces = []
    for p in range(1, len(integers)):
        spaces.append((integers[p] - integers[p-1]) > 1)
    edges = 2 + 2*sum(spaces)
    return edges

x = sum(cubes.groupby(by=['y','z']).apply(count_edges, 'x'))
y = sum(cubes.groupby(by=['x','z']).apply(count_edges, 'y'))
z = sum(cubes.groupby(by=['x','y']).apply(count_edges, 'z'))

x + y + z

array([64])

## Part 2

Count the exterior faces only.

In [4]:
def count_edges(cubes, axis):
    edges = 2
    return edges

x = sum(cubes.groupby(by=['y','z']).apply(count_edges, 'x'))
y = sum(cubes.groupby(by=['x','z']).apply(count_edges, 'y'))
z = sum(cubes.groupby(by=['x','y']).apply(count_edges, 'z'))

x + y + z

50

That's not right... need to consider whether each face is in a complete internal bubble, or not. The shape of the bubble could be pretty elaborate...